In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
import zipfile
from keras.metrics import Precision,Recall
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [ ]:
with zipfile.ZipFile("/content/Dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("Dataset")

In [ ]:
test_data = image_dataset_from_directory("/content/Dataset/Dataset/Test",image_size = (224,224),batch_size = 128*2)


Found 80362 files belonging to 2 classes.


In [ ]:
train_data = image_dataset_from_directory("/content/Dataset/Dataset/Train",image_size = (224,224),batch_size = 50)


Found 120542 files belonging to 2 classes.


In [ ]:
conv_base = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet'
)
conv_base.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [ ]:
inputs = keras.Input(shape = (224,224,3))
x = keras.applications.resnet50.preprocess_input(inputs)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256,activation = "relu")(x)
x = layers.Dense(128,activation = "relu")(x)
x  =layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model  = keras.Model(inputs = inputs,outputs = outputs)

In [ ]:
model.compile(optimizer="adam" ,loss ="binary_crossentropy",metrics =["accuracy", Precision(name='precision'), Recall(name='recall')])

In [ ]:
earlystop_cb = EarlyStopping(
    monitor='loss',
    patience=5,              # Wait for 5 epochs without improvement
    restore_best_weights=True,  # Load best weights after stopping
    verbose=1
)

In [ ]:
model.fit(train_data,epochs = 50,callbacks = [earlystop_cb])


Epoch 1/50
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 74s 26ms/step - accuracy: 0.8540 - loss: 0.3838 - precision: 0.6803 - recall: 0.5638
Epoch 2/50
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 55s 23ms/step - accuracy: 0.8719 - loss: 0.3303 - precision: 0.7574 - recall: 0.5483
Epoch 3/50
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 54s 23ms/step - accuracy: 0.8731 - loss: 0.3280 - precision: 0.7625 - recall: 0.5489
Epoch 4/50
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 54s 23ms/step - accuracy: 0.8730 - loss: 0.3265 - precision: 0.7639 - recall: 0.5472
Epoch 5/50
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 55s 23ms/step - accuracy: 0.8744 - loss: 0.3241 - precision: 0.7691 - recall: 0.5501
Epoch 6/50
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 54s 23ms/step - accuracy: 0.8757 - loss: 0.3227 - precision: 0.7730 - recall: 0.5540
Epoch 7/50
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 54s 23ms/step - accuracy: 0.8778 - loss: 0.3201 - precision: 0.7821 - recall: 0.5566
Epoch 8/50
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 54s 23ms/step - accuracy: 0.8785 - loss: 0.3179 - precision: 0.7853 -

In [ ]:
all_labels = []
for i,l in test_data:
  all_labels.extend(l.numpy())

In [ ]:
clean_labels = [int(x) for x in all_labels]

In [ ]:
y_pred = model.predict(test_data)

314/314 ━━━━━━━━━━━━━━━━━━━━ 40s 102ms/step


In [ ]:
y_pred_labels = (y_pred > 0.5).astype(int).flatten()


In [ ]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,accuracy_score

In [ ]:
accuracy_score(clean_labels,y_pred_labels)

0.6634478982603719

In [ ]:
recall_score(clean_labels,y_pred_labels)

0.2208732057416268

In [ ]:
precision_score(clean_labels,y_pred_labels)

0.20850271002710027

In [ ]:
f1_score(clean_labels,y_pred_labels)

0.21450975836431227

In [ ]:
confusion_matrix(clean_labels,y_pred_labels)

array([[49623, 14019],
       [13027,  3693]])